In [2]:
import pandas as pd
import numpy as np

In [3]:
# Reading CSV from link
def read_csv_from_link(url):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    df = pd.read_csv(path,delimiter="\t",error_bad_lines=False, header=None)
    return df

In [4]:
# Loading All Data
malayalam_train = read_csv_from_link('https://drive.google.com/file/d/13JCCr-IjZK7uhbLXeufptr_AxvsKinVl/view?usp=sharing')
malayalam_dev = read_csv_from_link('https://drive.google.com/file/d/1J0msLpLoM6gmXkjC6DFeQ8CG_rrLvjnM/view?usp=sharing')
malayalam_test = read_csv_from_link('https://drive.google.com/file/d/1waRFe4yTG8TMkMruICaavd9JH0xiO_rb/view?usp=sharing')

In [5]:
# Tamil Preprocess
malayalam_train = malayalam_train.iloc[:, 0:2]
malayalam_train = malayalam_train.rename(columns={0: "text", 1: "label"})

malayalam_dev = malayalam_dev.iloc[:, 0:2]
malayalam_dev = malayalam_dev.rename(columns={0: "text", 1: "label"})

malayalam_test = malayalam_test.iloc[:, 0:1]
malayalam_test = malayalam_test.rename(columns={0: "text"})

# Stats
malayalam_train['label'] = pd.Categorical(malayalam_train.label)
malayalam_dev['label'] = pd.Categorical(malayalam_dev.label)

### Load Model

In [6]:
import torch

In [7]:
print("GPU Nos: {}".format(torch.cuda.device_count()))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

# Change Device - CPU/GPU-0/GPU-1
torch.cuda.set_device(0)
device = 'cuda'
device = device if torch.cuda.is_available() else 'cpu'

GPU Nos: 2
Tesla P100-PCIE-12GB
Tesla P100-PCIE-16GB


## Enter Path of Saved model here in torch.load()

In [8]:
# Model Select
import os

from os import listdir
from os.path import isfile, join
[x for x in listdir('../../finetuned_berts/') if 'Malayalam' in x]

['XLMroberta_large_Malayalam',
 'Mbert_base_cased_Malayalam',
 'Distilbert_m_base_cased_Malayalam',
 'XLMroberta_base_Malayalam',
 'Indic_bert_Malayalam',
 'Indic_bert_Malayalam_weighted',
 'XLMroberta_base_Malayalam_weighted',
 'XLMroberta_large_Malayalam_weighted',
 'XLMroberta_custom_pretrained_Malayalam',
 'Distilbert_m_base_cased_Malayalam_weighted',
 'MURIL_cased_temp_Malayalam_weighted',
 'XLMroberta_custom_pretrained_Malayalam_weighted',
 'Mbert_base_cased_Malayalam_weighted',
 'MURIL_cased_temp_Malayalam']

In [9]:
models = []
tokenizers = []

In [10]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, XLMRobertaTokenizer

model_names = [
    'XLMroberta_base_Malayalam',
    'Mbert_base_cased_Malayalam',
]
tokenizers = [
    XLMRobertaTokenizer.from_pretrained('xlm-roberta-base'),
    BertTokenizer.from_pretrained('bert-base-multilingual-cased'),
]

for name in model_names:
    model = AutoModel.from_pretrained(os.path.join('../../finetuned_berts/', name))
    model.eval()
    models.append(model)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at ../../finetuned_berts/XLMroberta_base_Malayalam and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Dimension of BERT-ONLY fusion

In [11]:
lin_dim = np.sum([1024 if 'large' in name else 768 for name in model_names])
lin_dim

1536

In [12]:
n_models = len(models)

In [13]:
for model in models:
    for param in model.parameters():
        param.requires_grad = False

### Dataset Stuff

In [14]:
label_mapping = {
    'Not_offensive': 0, 
    'not-malayalam': 1, 
    'Offensive_Targeted_Insult_Group': 2, 
    'Offensive_Untargetede': 3, 
    'Offensive_Targeted_Insult_Individual': 4
}

# Collecting Text and Labels
train_batch_sentences = list(malayalam_train['text'])
train_batch_labels =  [label_mapping[x] for x in malayalam_train['label']]
dev_batch_sentences = list(malayalam_dev['text'])
dev_batch_labels =  [label_mapping[x] for x in malayalam_dev['label']]
test_batch_sentences = list(malayalam_test['text'])

# Convert to Tensor
train_encodings = [tokenizer(train_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt") for tokenizer in tokenizers]
train_labels = torch.tensor(train_batch_labels)
dev_encodings = [tokenizer(dev_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt") for tokenizer in tokenizers]
dev_labels = torch.tensor(dev_batch_labels)
test_encodings = [tokenizer(test_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt") for tokenizer in tokenizers]

In [15]:
# Dataset
from torch.utils.data import Dataset

class malayalam_Offensive_Dataset(Dataset):
    def __init__(self, encodings, labels = None):
        self.encodings = encodings
        self.labels = labels
        self.n_models = len(encodings)

    def __getitem__(self, idx):
        item = {}
        for i in range(self.n_models):
            item.update({key+'_'+str(i): torch.tensor(val[idx]) for key, val in self.encodings[i].items()})
        item['index'] = idx
        if self.labels != None:
            item['labels'] = torch.tensor(self.labels[idx])
        else:
            item['labels'] = torch.tensor(1)
        return item

    def __len__(self):
        return len(self.encodings[0]['input_ids'])

# Defining Datasets
train_dataset = malayalam_Offensive_Dataset(train_encodings, train_labels)
dev_dataset = malayalam_Offensive_Dataset(dev_encodings, dev_labels)
test_dataset = malayalam_Offensive_Dataset(test_encodings)

#### FUSION MODEL

In [16]:
import torch.nn.functional as F
import torch.nn as nn

# Basic Fully-Connected (Linear => BatchNorm => ReLU)
class BasicFC(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicFC, self).__init__()
        self.fc = nn.Linear(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm1d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class FusionNet(torch.nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(FusionNet, self).__init__()
        self.linear1_1 = BasicFC(D_in, H1)
        self.linear1_2 = BasicFC(H1, H2)
        self.linear1_3 = BasicFC(H2, H3)
        self.dp = nn.Dropout(0.1)
        self.linear2 = torch.nn.Linear(H3, D_out, bias = False)

    def forward(self, x):
        h_relu_1 = self.linear1_1(x)
        h_relu_2 = self.dp(self.linear1_2(h_relu_1))
        h_relu_3 = self.dp(self.linear1_3(h_relu_2))
        y_pred = self.linear2(h_relu_3)
        return y_pred

In [17]:
# Optimiser
from transformers import AdamW

In [18]:
## Loss Fn
XE_loss_function = nn.CrossEntropyLoss(reduction='mean').float()

In [19]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7, n_labels = 6):
        super().__init__()
        self.epsilon = epsilon
        self.n_labels = n_labels
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.n_labels).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()

F1_loss_function = F1_Loss(n_labels = 5).cuda()

In [20]:
use_f1_loss = False
if use_f1_loss:
    loss_function = F1_loss_function
else:
    loss_function = XE_loss_function

In [21]:
add_extra_embeds = [
    'cnn_128',
    #'sentbert',
    #'laser_ta',
    #'laser_en',
]

embeds_files = {
    'cnn_128': ['../../sentence_embeddings/cnn_emb_train_128_malayalam.npy', '../../sentence_embeddings/cnn_emb_dev_128_malayalam.npy', '../../sentence_embeddings/cnn_emb_test_128_malayalam.npy'],
    'laser_ta': ['../../sentence_embeddings/LASER_embeddings_malayalam_ml_train.npy', '../../sentence_embeddings/LASER_embeddings_malayalam_ml_dev.npy'],
    'laser_en': ['../../sentence_embeddings/LASER_embeddings_malayalam_en_train.npy', '../../sentence_embeddings/LASER_embeddings_malayalam_en_dev.npy'],
    'sentbert': ['../../sentence_embeddings/sentence_bert_malayalam_train.npy', '../../sentence_embeddings/sentence_bert_malayalam_dev.npy'],
}

train_embeddings = [np.load(embeds_files[embname][0]) for embname in add_extra_embeds]
dev_embeddings = [np.load(embeds_files[embname][1]) for embname in add_extra_embeds]
test_embeddings = [np.load(embeds_files[embname][2]) for embname in add_extra_embeds]

In [22]:
len_extra_embeds = len(add_extra_embeds)
len_extra_embeds

1

In [23]:
dim_extra_embeds = np.sum([train_embeddings[i].shape[1] for i in range(len_extra_embeds)])
dim_extra_embeds

128

In [1]:
# Final Input embeddings dimensions

In [24]:
print(lin_dim + dim_extra_embeds)

1664


### Train

In [25]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, f1_score
import copy

fusion_classifier = FusionNet(int(lin_dim + dim_extra_embeds), 1024, 256, 64, 5)
# Optimiser
optimizer = AdamW(fusion_classifier.parameters(), lr=1e-5)
mo,ex_em = "",""
for mod in model_names:
    mo += mod
for em in add_extra_embeds:
    ex_em += em
model_name = 'fusion_malayalam_'+mo+'_'+ex_em

best_model = None

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
fusion_classifier.to(device)
for model in models:
    model.to(device)
best_val_f1 = 0
count = 0

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

for epoch in range(100):
    fusion_classifier.train()
    print("==========================================================")
    print("Epoch {}".format(epoch))
    print("Train")
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])
        
        for i in range(len_extra_embeds):
            outputs_all.append(torch.Tensor(train_embeddings[i][batch['index'], :]).to(device))
        
        bert_output = torch.cat(outputs_all, dim = -1) 
        out = fusion_classifier(bert_output)
        loss = loss_function(out, labels)
        loss.backward()
        optimizer.step()
    
    print("Dev")
    dev_preds = []
    fusion_classifier.eval()
    total_val_loss = 0
    with torch.set_grad_enabled(False):
        for batch in tqdm(dev_loader):
            outputs_all = []
            for i in range(n_models):
                model = models[i]
                input_ids = batch['input_ids'+'_'+str(i)].to(device)
                attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                outputs_all.append(outputs[1])
                
            for i in range(len_extra_embeds):
                outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

            bert_output = torch.cat(outputs_all, dim = -1) 
            out = fusion_classifier(bert_output)
            loss = loss_function(out, labels)
            total_val_loss += loss.item()/len(dev_loader)
            
            for logits in out.cpu().numpy():
                dev_preds.append(np.argmax(logits))
    
    y_true = dev_batch_labels
    y_pred = dev_preds
    target_names = label_mapping.keys()
    report = classification_report(y_true, y_pred, target_names=target_names)
    val_f1 = f1_score(y_true, y_pred, average='macro')
    
    if val_f1 > best_val_f1:
        PATH = '../../finetuned_models/' + model_name + '.pth'
        torch.save(fusion_classifier.state_dict(), PATH)
        best_model = copy.deepcopy(fusion_classifier)
        best_val_f1 = val_f1
        count = 0
    else:
        count += 1
    
    print(report)
    print("Epoch {}, Val Loss = {}, Val F1 = {}, Best Val f1 = {}, stagnant_t = {}".format(epoch, total_val_loss, val_f1, best_val_f1, count))
    if count == 5:
        print("No increase for 5 epochs, Stopping ...")
        break

Epoch 0
Train


/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()



Dev



                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.98      0.98      1779
                       not-malayalam       0.88      0.93      0.90       163
     Offensive_Targeted_Insult_Group       0.82      0.69      0.75        13
               Offensive_Untargetede       0.57      0.65      0.60        20
Offensive_Targeted_Insult_Individual       0.86      0.50      0.63        24

                            accuracy                           0.97      1999
                           macro avg       0.82      0.75      0.77      1999
                        weighted avg       0.97      0.97      0.97      1999

Epoch 0, Val Loss = 0.7450550622940062, Val F1 = 0.7744316339493207, Best Val f1 = 0.7744316339493207, stagnant_t = 0
Epoch 1
Train



Dev


...


                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.99      0.98      1779
                       not-malayalam       0.90      0.91      0.91       163
     Offensive_Targeted_Insult_Group       0.75      0.69      0.72        13
               Offensive_Untargetede       0.76      0.65      0.70        20
Offensive_Targeted_Insult_Individual       0.93      0.54      0.68        24

                            accuracy                           0.97      1999
                           macro avg       0.87      0.76      0.80      1999
                        weighted avg       0.97      0.97      0.97      1999

Epoch 8, Val Loss = 0.1685241443514824, Val F1 = 0.7998406631569812, Best Val f1 = 0.7999497065456469, stagnant_t = 3
Epoch 9
Train



Dev



                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.99      0.98      1779
                       not-malayalam       0.91      0.90      0.90       163
     Offensive_Targeted_Insult_Group       0.75      0.69      0.72        13
               Offensive_Untargetede       0.81      0.65      0.72        20
Offensive_Targeted_Insult_Individual       0.92      0.50      0.65        24

                            accuracy                           0.97      1999
                           macro avg       0.87      0.75      0.80      1999
                        weighted avg       0.97      0.97      0.97      1999

Epoch 9, Val Loss = 0.14831925016641612, Val F1 = 0.7957965517965518, Best Val f1 = 0.7999497065456469, stagnant_t = 4
Epoch 10
Train



Dev



                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.99      0.98      1779
                       not-malayalam       0.90      0.91      0.91       163
     Offensive_Targeted_Insult_Group       0.75      0.69      0.72        13
               Offensive_Untargetede       0.76      0.65      0.70        20
Offensive_Targeted_Insult_Individual       0.92      0.50      0.65        24

                            accuracy                           0.97      1999
                           macro avg       0.86      0.75      0.79      1999
                        weighted avg       0.97      0.97      0.97      1999

Epoch 10, Val Loss = 0.14941174986958516, Val F1 = 0.7926731908158576, Best Val f1 = 0.7999497065456469, stagnant_t = 5
No increase for 5 epochs, Stopping ...


#### Final Model predictions on Dev

In [27]:
dev_preds = []
dev_probs = []
best_model.eval()
with torch.set_grad_enabled(False):
    for batch in tqdm(dev_loader):
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])

        for i in range(len_extra_embeds):
            outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

        bert_output = torch.cat(outputs_all, dim = -1) 
        out = best_model(bert_output)

        for logits in out.cpu().numpy():
            dev_preds.append(np.argmax(logits))
        for logits in out.cpu().numpy():
            dev_probs.append(np.exp(logits)/np.sum(np.exp(logits)))

y_true = dev_batch_labels
y_pred = dev_preds
target_names = label_mapping.keys()
report = classification_report(y_true, y_pred, target_names=target_names)
val_f1 = f1_score(y_true, y_pred, average='macro')

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


In [28]:
print(report)

                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.99      0.98      1779
                       not-malayalam       0.90      0.92      0.91       163
     Offensive_Targeted_Insult_Group       0.75      0.69      0.72        13
               Offensive_Untargetede       0.76      0.65      0.70        20
Offensive_Targeted_Insult_Individual       0.93      0.54      0.68        24

                            accuracy                           0.97      1999
                           macro avg       0.86      0.76      0.80      1999
                        weighted avg       0.97      0.97      0.97      1999



In [32]:
print(model_name)

fusion_malayalam_XLMroberta_base_MalayalamMbert_base_cased_Malayalam_cnn_128


In [31]:
#### SAVE PROBS
np.save('dev_probs/dev_preds_'+model_name+'.npy', dev_probs)

#### Final Model predictions on Test

In [34]:
best_model.eval()
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_preds = []
test_probs = []
total_val_loss = 0
with torch.set_grad_enabled(False):
    for batch in tqdm(test_loader):
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])

        for i in range(len_extra_embeds):
            if len(test_embeddings[i][batch['index'], :]) == 128:
                outputs_all.append(torch.Tensor(test_embeddings[i][batch['index'], :]).unsqueeze(0).to(device))
            else:
                outputs_all.append(torch.Tensor(test_embeddings[i][batch['index'], :]).to(device))
    
        bert_output = torch.cat(outputs_all, dim = -1) 
        out = best_model(bert_output)
 
        for logits in out.cpu().numpy():
            test_preds.append(np.argmax(logits))
        for logits in out.cpu().numpy():
            test_probs.append(np.exp(logits)/np.sum(np.exp(logits)))

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [36]:
#### SAVE PROBS
np.save('test_probs/test_preds_'+model_name+'.npy', test_probs)

#### Save config in dict for predictions and ensemble

In [56]:
dict = {}
dict['pth_name'] = model_name
dict['model'] = model_names
dict['pretrained_keys'] = ['xlm-roberta-base','xlm-roberta-base','bert-base-multilingual-cased']
dict['extra_embeds'] = add_extra_embeds
dict['length'] = lin_dim + dim_extra_embeds

In [57]:
model_name_tot = 'dicts/'+model_name

In [59]:
import pickle
with open(str(model_name_tot)+'.pickle', 'wb') as handle:
    pickle.dump(dict, handle)

### Save Preds - Get predictions from individual model

In [26]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, f1_score

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Dataloaders
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)
fusion_classifier.load_state_dict(torch.load('../../finetuned_models/' + model_name + '.pth'))
fusion_classifier.eval()

dev_preds = []
with torch.set_grad_enabled(False):
    for batch in tqdm(dev_loader):
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])

        for i in range(len_extra_embeds):
                outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

        bert_output = torch.cat(outputs_all, dim = -1) 
        out = fusion_classifier(bert_output)

        for logits in out.cpu().numpy():
            dev_preds.append(np.argmax(logits))

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [27]:
y_true = dev_batch_labels
y_pred = dev_preds
target_names = label_mapping.keys()
report = classification_report(y_true, y_pred, target_names=target_names)

In [28]:
print(report)

                                      precision    recall  f1-score   support

                       Not_offensive       0.97      0.99      0.98      1779
                       not-malayalam       0.90      0.86      0.88       163
     Offensive_Targeted_Insult_Group       0.80      0.62      0.70        13
               Offensive_Untargetede       0.80      0.60      0.69        20
Offensive_Targeted_Insult_Individual       0.92      0.50      0.65        24

                            accuracy                           0.96      1999
                           macro avg       0.88      0.71      0.78      1999
                        weighted avg       0.96      0.96      0.96      1999



#### Final Model predictions on Test

In [29]:
np.savetxt("../../dev_preds/" + model_name + ".csv", dev_preds, delimiter=",")